In [11]:
import spacy
import pickle
import random

ModuleNotFoundError: ignored

In [12]:
!pip install PyMuPDF

     |████████████████████████████████| 6.4 MB 12.1 MB/s 


In [13]:
import sys, fitz

In [2]:
!git clone https://github.com/laxmimerit/Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python.git

Cloning into 'Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [3]:
train_data = pickle.load(open('/content/Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python/train_data.pkl', 'rb'))

In [4]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [5]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)  
    

In [6]:
train_model(train_data)

Statring iteration 0
{'ner': 16025.374306274563}
Statring iteration 1
{'ner': 6970.50497810004}
Statring iteration 2
{'ner': 8099.69608098048}
Statring iteration 3
{'ner': 5354.330367719865}
Statring iteration 4
{'ner': 7095.667677543774}
Statring iteration 5
{'ner': 5895.383440774901}
Statring iteration 6
{'ner': 5307.975516742968}
Statring iteration 7
{'ner': 4103.556920135014}
Statring iteration 8
{'ner': 4421.364590935201}
Statring iteration 9
{'ner': 4553.19716159358}


In [7]:
nlp.to_disk('nlp_model')

In [8]:
nlp_model = spacy.load('nlp_model')

In [9]:
train_data[0][0]

'Rahul Bollu Software Engineer - Disney  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Rahul-Bollu/dc40f5ce78045741  • Over 3.5 years of experience in implementing organization DevOps strategy in various environments of Linux and windows servers along with adopting cloud strategies based on Amazon Web Services. • Experience in Cloud Technologies like Amazon Web Services (AWS) VPC, EC2, S3, ELB, IAM, Auto Scaling, Route 53, SQS, SNS, RDS, Cloud Watch, Dynamo DB. • Utilized Cloud Watch to monitor AWS resources to set alarms for notification, and to monitor logs for a better operation of the system. • Experience working with automated build platforms/continuous integration using DevOps architecture. • Implementing DevOps tools like Ansible as configuration management for Continuous Integration and Continuous Deployment with build tools using Maven on Cloud Infrastructure using AWS. • Experience on version control tool GIT- Creating branches, tracking changes, maintaining the his

In [10]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Rahul Bollu
DESIGNATION                   - Software Engineer
LOCATION                      - Hyderabad
DESIGNATION                   - Software Engineer
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
DEGREE                        - Bachelor Of Science
COLLEGE NAME                  - Vaughn College of Aeronautics and Technology
SKILLS                        - AWS (3 years), Tomcat, Ansible, git, LAMP, docker, jenkins, Maven, Jira  ADDITIONAL INFORMATION  Technical Skills:  Cloud Technologies: AWS  Operating Systems: Linux, Windows. Version Control Systems: GIT Automated Build Tools: Maven Continuous Integration: Jenkins    Scripting Languages: Shell Scripting Configuration Management: Ansible. Container service: Docker


In [14]:
fname = '/content/Resume-and-CV-Summarization-and-Parsing-with-Spacy-in-Python/Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [15]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
DESIGNATION                   - AI / Machine Learning
COMPANIES WORKED AT           - Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, browsing with  Microsoft Edge and making purchases at the Xbox Store, the Windows Store and the Microsoft  Store. Plus, user can pick up bonus points for taking daily quizzes and tours on the Microsoft  rewards website. Rewards live dashboards gives a live picture of usage world-wide and by  markets like US, Canada, Australia, new user registration count, top/bottom performing rewards  offers, orders stats and weekly trends of user activities, orders and new user registrations. the  PBI tiles gets refreshed in different frequencies starting from 5 seconds to 30 minutes.  Technology/Tools use